In [1]:
%matplotlib inline


Sequence-to-Sequence Modeling with nn.Transformer and TorchText
===============================================================

This is a tutorial on how to train a sequence-to-sequence model
that uses the
`nn.Transformer <https://pytorch.org/docs/master/nn.html?highlight=nn%20transformer#torch.nn.Transformer>`__ module.

PyTorch 1.2 release includes a standard transformer module based on the
paper `Attention is All You
Need <https://arxiv.org/pdf/1706.03762.pdf>`__. The transformer model
has been proved to be superior in quality for many sequence-to-sequence
problems while being more parallelizable. The ``nn.Transformer`` module
relies entirely on an attention mechanism (another module recently
implemented as `nn.MultiheadAttention <https://pytorch.org/docs/master/nn.html?highlight=multiheadattention#torch.nn.MultiheadAttention>`__) to draw global dependencies
between input and output. The ``nn.Transformer`` module is now highly
modularized such that a single component (like `nn.TransformerEncoder <https://pytorch.org/docs/master/nn.html?highlight=nn%20transformerencoder#torch.nn.TransformerEncoder>`__
in this tutorial) can be easily adapted/composed.

![](../_static/img/transformer_architecture.jpg)





Define the model
----------------




In this tutorial, we train ``nn.TransformerEncoder`` model on a
language modeling task. The language modeling task is to assign a
probability for the likelihood of a given word (or a sequence of words)
to follow a sequence of words. A sequence of tokens are passed to the embedding
layer first, followed by a positional encoding layer to account for the order
of the word (see the next paragraph for more details). The
``nn.TransformerEncoder`` consists of multiple layers of
`nn.TransformerEncoderLayer <https://pytorch.org/docs/master/nn.html?highlight=transformerencoderlayer#torch.nn.TransformerEncoderLayer>`__. Along with the input sequence, a square
attention mask is required because the self-attention layers in
``nn.TransformerEncoder`` are only allowed to attend the earlier positions in
the sequence. For the language modeling task, any tokens on the future
positions should be masked. To have the actual words, the output
of ``nn.TransformerEncoder`` model is sent to the final Linear
layer, which is followed by a log-Softmax function.




In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

``PositionalEncoding`` module injects some information about the
relative or absolute position of the tokens in the sequence. The
positional encodings have the same dimension as the embeddings so that
the two can be summed. Here, we use ``sine`` and ``cosine`` functions of
different frequencies.




In [3]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

Load and batch data
-------------------




The training process uses Wikitext-2 dataset from ``torchtext``. The
vocab object is built based on the train dataset and is used to numericalize
tokens into tensors. Starting from sequential data, the ``batchify()``
function arranges the dataset into columns, trimming off any tokens remaining
after the data has been divided into batches of size ``batch_size``.
For instance, with the alphabet as the sequence (total length of 26)
and a batch size of 4, we would divide the alphabet into 4 sequences of
length 6:

\begin{align}\begin{bmatrix}
  \text{A} & \text{B} & \text{C} & \ldots & \text{X} & \text{Y} & \text{Z}
  \end{bmatrix}
  \Rightarrow
  \begin{bmatrix}
  \begin{bmatrix}\text{A} \\ \text{B} \\ \text{C} \\ \text{D} \\ \text{E} \\ \text{F}\end{bmatrix} &
  \begin{bmatrix}\text{G} \\ \text{H} \\ \text{I} \\ \text{J} \\ \text{K} \\ \text{L}\end{bmatrix} &
  \begin{bmatrix}\text{M} \\ \text{N} \\ \text{O} \\ \text{P} \\ \text{Q} \\ \text{R}\end{bmatrix} &
  \begin{bmatrix}\text{S} \\ \text{T} \\ \text{U} \\ \text{V} \\ \text{W} \\ \text{X}\end{bmatrix}
  \end{bmatrix}\end{align}

These columns are treated as independent by the model, which means that
the dependence of ``G`` and ``F`` can not be learned, but allows more
efficient batch processing.




In [4]:
#train_data.type, val_data.shape, test_data.shape

In [5]:
#pre_train[-5:]

In [6]:
import torchtext
TEXT = torchtext.data.Field(tokenize=list,
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)

dico_file = open("dico_fr.txt", 'r')
dico_string = dico_file.read()
pre_train = dico_string.split('\n')
train = [x.rjust(25) for x in pre_train]


TEXT.build_vocab(train)
train_txt = TEXT.numericalize(train)
val_txt = test_txt = train_txt
print(TEXT.numericalize(train))
print('long', len(TEXT.vocab.stoi))

tensor([[ 4,  4,  4,  ...,  4,  4,  4],
        [ 4,  4,  4,  ...,  4,  4,  4],
        [ 4,  4,  4,  ...,  4,  4,  4],
        ...,
        [ 4,  4,  6,  ..., 24, 15,  4],
        [ 4,  4,  6,  ..., 15, 18,  4],
        [ 8, 13,  8,  ..., 18,  6,  4]])
long 66


In [7]:

train_txt = torch.transpose(train_txt, 0, 1)
train_txt = torch.reshape(train_txt, (100,84133))
val_txt = train_txt
test_txt = train_txt

In [8]:
converter = np.vectorize(lambda x: TEXT.vocab.itos[x])

In [9]:
converter(train_txt)

array([[' ', ' ', ' ', ..., ' ', ' ', ' '],
       [' ', ' ', ' ', ..., ' ', ' ', ' '],
       ['a', 'f', 'f', ..., '¢', 'm', 'e'],
       ...,
       [' ', ' ', ' ', ..., ' ', ' ', ' '],
       [' ', ' ', ' ', ..., 'v', 'i', 'r'],
       ['e', 'v', 'o', ..., ' ', ' ', ' ']], dtype='<U1')

In [10]:
print(TEXT.numericalize(train))
print('long', len(TEXT.vocab.stoi))

tensor([[ 4,  4,  4,  ...,  4,  4,  4],
        [ 4,  4,  4,  ...,  4,  4,  4],
        [ 4,  4,  4,  ...,  4,  4,  4],
        ...,
        [ 4,  4,  6,  ..., 24, 15,  4],
        [ 4,  4,  6,  ..., 15, 18,  4],
        [ 8, 13,  8,  ..., 18,  6,  4]])
long 66


In [11]:

from torchtext.data.utils import get_tokenizer
'''TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)'''
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    #data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    print(nbatch)
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 25
eval_batch_size = 25
train_data = batchify(train_txt, batch_size)
#train_data = torch.transpose(train_data, 0, 1)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

4
4
4


In [12]:
train_txt.shape

torch.Size([100, 84133])

In [13]:
train_data.shape

torch.Size([336532, 25])

In [14]:

#train_data = torch.reshape(train_data, (84133,100))
#train_data.shape
#val_data = train_data
#test_data = train_data

In [15]:
#train_data = torch.transpose(train_data, 0, 1)

Functions to generate input and target sequence
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~




In [16]:
'''dico_file = open("dico_fr.txt", 'r')
dico_string = dico_file.read()
dico_list = dico_string.split('\n')
lists_of_letters = []
for x in dico_list:
    word_list = [ord(i)-ord(' ') for i in x.rjust(25)]
    lists_of_letters.append(word_list)    
train_data = torch.tensor(lists_of_letters[:100])
val_data = train_data
test_data = train_data
print(train_data)'''

'dico_file = open("dico_fr.txt", \'r\')\ndico_string = dico_file.read()\ndico_list = dico_string.split(\'\n\')\nlists_of_letters = []\nfor x in dico_list:\n    word_list = [ord(i)-ord(\' \') for i in x.rjust(25)]\n    lists_of_letters.append(word_list)    \ntrain_data = torch.tensor(lists_of_letters[:100])\nval_data = train_data\ntest_data = train_data\nprint(train_data)'

``get_batch()`` function generates the input and target sequence for
the transformer model. It subdivides the source data into chunks of
length ``bptt``. For the language modeling task, the model needs the
following words as ``Target``. For example, with a ``bptt`` value of 2,
we’d get the following two Variables for ``i`` = 0:

![](../_static/img/transformer_input_target.png)


It should be noted that the chunks are along dimension 0, consistent
with the ``S`` dimension in the Transformer model. The batch dimension
``N`` is along dimension 1.




In [17]:
bptt = 25
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

Initiate an instance
--------------------




The model is set up with the hyperparameter below. The vocab size is
equal to the length of the vocab object.




In [18]:
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

Run the model
-------------




`CrossEntropyLoss <https://pytorch.org/docs/master/nn.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss>`__
is applied to track the loss and
`SGD <https://pytorch.org/docs/master/optim.html?highlight=sgd#torch.optim.SGD>`__
implements stochastic gradient descent method as the optimizer. The initial
learning rate is set to 5.0. `StepLR <https://pytorch.org/docs/master/optim.html?highlight=steplr#torch.optim.lr_scheduler.StepLR>`__ is
applied to adjust the learn rate through epochs. During the
training, we use
`nn.utils.clip_grad_norm\_ <https://pytorch.org/docs/master/nn.html?highlight=nn%20utils%20clip_grad_norm#torch.nn.utils.clip_grad_norm_>`__
function to scale all the gradient together to prevent exploding.




In [19]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

Loop over epochs. Save the model if the validation loss is the best
we've seen so far. Adjust the learning rate after each epoch.



In [20]:
best_val_loss = float("inf")
epochs = 2 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

/home/pierre/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |   200/13461 batches | lr 5.00 | ms/batch 40.51 | loss  2.33 | ppl    10.23
| epoch   1 |   400/13461 batches | lr 5.00 | ms/batch 42.93 | loss  1.10 | ppl     3.00
| epoch   1 |   600/13461 batches | lr 5.00 | ms/batch 35.62 | loss  1.05 | ppl     2.84
| epoch   1 |   800/13461 batches | lr 5.00 | ms/batch 32.14 | loss  1.05 | ppl     2.85
| epoch   1 |  1000/13461 batches | lr 5.00 | ms/batch 29.64 | loss  1.00 | ppl     2.73
| epoch   1 |  1200/13461 batches | lr 5.00 | ms/batch 30.37 | loss  0.97 | ppl     2.65
| epoch   1 |  1400/13461 batches | lr 5.00 | ms/batch 25.06 | loss  0.99 | ppl     2.70
| epoch   1 |  1600/13461 batches | lr 5.00 | ms/batch 25.82 | loss  0.97 | ppl     2.65
| epoch   1 |  1800/13461 batches | lr 5.00 | ms/batch 22.88 | loss  0.95 | ppl     2.60
| epoch   1 |  2000/13461 batches | lr 5.00 | ms/batch 24.63 | loss  0.97 | ppl     2.64
| epoch   1 |  2200/13461 batches | lr 5.00 | ms/batch 23.93 | loss  0.95 | ppl     2.59
| epoch   1 |  2400/1

| epoch   2 |  4800/13461 batches | lr 4.51 | ms/batch 22.92 | loss  0.92 | ppl     2.52
| epoch   2 |  5000/13461 batches | lr 4.51 | ms/batch 29.85 | loss  0.89 | ppl     2.44
| epoch   2 |  5200/13461 batches | lr 4.51 | ms/batch 26.72 | loss  0.92 | ppl     2.50
| epoch   2 |  5400/13461 batches | lr 4.51 | ms/batch 24.89 | loss  0.86 | ppl     2.37
| epoch   2 |  5600/13461 batches | lr 4.51 | ms/batch 25.80 | loss  0.96 | ppl     2.61
| epoch   2 |  5800/13461 batches | lr 4.51 | ms/batch 23.83 | loss  0.93 | ppl     2.54
| epoch   2 |  6000/13461 batches | lr 4.51 | ms/batch 24.27 | loss  0.92 | ppl     2.51
| epoch   2 |  6200/13461 batches | lr 4.51 | ms/batch 26.04 | loss  0.94 | ppl     2.56
| epoch   2 |  6400/13461 batches | lr 4.51 | ms/batch 25.77 | loss  0.91 | ppl     2.48
| epoch   2 |  6600/13461 batches | lr 4.51 | ms/batch 20.65 | loss  0.93 | ppl     2.54
| epoch   2 |  6800/13461 batches | lr 4.51 | ms/batch 25.65 | loss  0.95 | ppl     2.57
| epoch   2 |  7000/1

In [21]:
best_model

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=200, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=200, out_features=200, bias=True)
        (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=20

In [22]:
train_data.shape

torch.Size([336532, 25])

In [23]:
mot = train_data[:,11:12].clone()
mot.shape

torch.Size([336532, 1])

In [24]:
mot = mot[100:200]

In [25]:
for x in converter(mot):
    for i in x:
        print(i)

 
 
 
 
 
 
 
 
 
 
 
 
e
x
p
o
r
t
e
r
a
i
s
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
x
p
o
r
t
e
r
a
i
t
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
x
p
o
r
t
e
r
a
s
 
 
 
 
 
 
 
 
 
 
 
 
 
e
x
p
o
r
t
Ã
¨
r
e
n
t
 
 


In [28]:
print("mot",converter(mot))
for i in range(0,len(mot)-1):
    prediction = best_model(mot).argmax(axis=2)
    if converter(prediction)[ i] != [' ']:
        print(converter(mot)[i + 1], 'par', converter(prediction)[ i])
        mot[ i+1] = prediction[ i]
        print("mot",converter(mot))
        print("pred" ,converter(prediction))
    #print("mot",converter(mot))
print("mot",converter(mot))
    

mot [[' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 ['e']
 ['n']
 ['s']
 ['o']
 ['l']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 ['e']
 ['n']
 ['t']
 ['r']
 ['e']
 ['n']
 ['t']
 ['r']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']]
['n'] par ['n']
mot [[' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 ['e']
 ['n']
 ['s']
 ['o']
 ['l']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 ['e']
 ['n']
 [

['a'] par ['a']
mot [[' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 ['e']
 ['n']
 ['s']
 ['o']
 ['l']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 ['e']
 ['n']
 ['t']
 ['r']
 ['e']
 ['n']
 ['t']
 ['r']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']]
pred [[' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 ['n']
 ['s']
 [' ']
 ['l']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['n']
 ['t']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 [' ']
 ['n']
 ['t']
 

In [72]:
mot = torch.tensor([[22]])
print("mot",converter(mot))
for i in range(0,25):
    prediction = best_model(mot).argmax(axis=2)
    print("pred" ,converter(prediction))
    #print(converter(mot)[i + 1], 'par', converter(prediction)[ i])
    mot = torch.cat((mot, torch.tensor([[prediction[i]]])))
    print("mot",converter(mot))
    print("pred" ,converter(prediction))
    #print("mot",converter(mot))
print("mot",converter(mot))
    

mot [['b']]
pred [['a']]
mot [['b']
 ['a']]
pred [['a']]
pred [['a']
 ['i']]
mot [['b']
 ['a']
 ['i']]
pred [['a']
 ['i']]
pred [['a']
 ['i']
 ['e']]
mot [['b']
 ['a']
 ['i']
 ['e']]
pred [['a']
 ['i']
 ['e']]
pred [['a']
 ['i']
 ['e']
 ['r']]
mot [['b']
 ['a']
 ['i']
 ['e']
 ['r']]
pred [['a']
 ['i']
 ['e']
 ['r']]
pred [['a']
 ['i']
 ['e']
 ['r']
 ['a']]
mot [['b']
 ['a']
 ['i']
 ['e']
 ['r']
 ['a']]
pred [['a']
 ['i']
 ['e']
 ['r']
 ['a']]
pred [['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']]
mot [['b']
 ['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']]
pred [['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']]
pred [['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']]
mot [['b']
 ['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']]
pred [['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']]
pred [['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['r']]
mot [['b']
 ['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['r']]
pred [['a']
 ['i']
 ['e']
 ['r']
 ['a']
 ['i']
 ['e']
 ['r']]
pred [['a']
 ['i']
 ['e']
 ['r']
 ['a']


In [27]:
best_model(mot)

tensor([[[-2.6785, -2.4773, -2.3849,  ..., -2.4287, -2.4497, -2.3458]],

        [[-2.6785, -2.4773, -2.3849,  ..., -2.4287, -2.4497, -2.3458]],

        [[-2.6785, -2.4773, -2.3849,  ..., -2.4287, -2.4497, -2.3458]],

        ...,

        [[-2.3677, -2.7159, -2.6809,  ..., -2.6984, -2.7846, -2.4202]],

        [[-2.8605, -2.9349, -2.9208,  ..., -3.0034, -3.2915, -3.0140]],

        [[-2.4342, -2.3395, -2.2136,  ..., -2.3178, -2.5192, -2.1722]]],
       grad_fn=<AddBackward0>)

In [ ]:
mot

In [ ]:
converter(mot)

In [ ]:
converter(train_data)

In [ ]:
#train_out = best_model(train_data).argmax(axis=2)

In [ ]:
converter(best_model(mot).argmax(axis=2))

In [ ]:
#train_out

In [ ]:
'''for i, line in enumerate(converter(train_out.numpy())):
    print(''.join(line), pre_train[i], i)'''

Evaluate the model with the test dataset
-------------------------------------

Apply the best model to check the result with the test dataset.



In [ ]:
test_loss = evaluate(best_model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)